In [1]:
#Imports
import json
import numpy as np
import pandas as pd
import csv
import os

In [2]:
#encodes a dictionary into a jason
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)


def jasonGenerator(path,results,photo_name):
    """ Generates a .jason file in the path given folder from the a dictionary """
    with open(path + '/analysis_result_' + photo_name + '.json',"w") as miarch:
        miarch.write(json.dumps(results, cls=NumpyEncoder))

In [3]:
def jasonReader(path):
    """ Reads the .jason generated with jasonGenartor the given path"""
    with open(path,'r') as miarch:
        loaded_dict = json.loads(miarch.read())
    return loaded_dict

In [4]:
def slicesParser_max_peaksI(dictres):
#extract all the slices data
    allSlices = dictres['slices']
    #seting columns names
    column_names = ['transient' + str(x) for x in range(0,len(allSlices[0]['max_peaks_intensities']))]    
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    for i in range(0,len(allSlices)):
        df_sum.loc[i] = allSlices[i]['max_peaks_intensities'][0:] 
    return df_sum

In [5]:
def slicesParser_halfpeakstime(dictres):
    #extract all the slices data
    allSlices = dictres['slices']
    
    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['times_to_half_peaks']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['times_to_half_peaks']
    return df_sum

In [6]:
def slicesParser_amplitudes(dictres):
    #extract all the slices data
    allSlices = dictres['slices']
    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['amplitudes']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['amplitudes']

    return df_sum

In [7]:
def slicesParser_min_peaksI(dictres):
    #extract all the slices data
    allSlices = dictres['slices']
    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['min_peaks_intensities']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['min_peaks_intensities']

    return df_sum

In [8]:
def slicesParser_peaktime(dictres):
    #extract all the slices data
    allSlices = dictres['slices']

    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['times_to_peaks']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['times_to_peaks']

    return df_sum

In [17]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

# Calcula el alternance ratio

AR_list = []
path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/jsons/'
jsons_list = os.listdir(path)
for file in jsons_list:
    path_file = path + file
    dic = jasonReader(path_file)
    slices = dic['slices']
    i = 0
    df = pd.DataFrame(columns=['transient1','transient2','transient_max'])
    print (file)
    for i in range(0,33):
        try:
            amps = slices[i]['amplitudes'][0:2]
            amps.append(max(amps))
            i += 1
            df.loc[len(df)] = amps
        except IndexError:
            pass
    df.to_csv(path + file + 'max_amp.csv', sep='\t')
print('Done')

In [ ]:
# Guarda en tablas separadas los datos de cada json por parámetro

param_list = ['max_peaks_positions','min_peaks_positions','max_peaks_intensities','min_peaks_intensities','amplitudes','times_to_peaks','times_to_half_peaks']

for param in param_list:
    os.makedirs(path2 + param + '/')
    path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/jsons/'
    path2 = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/'
    jsons_list = os.listdir(path)
    for file in jsons_list:
        path_file = path + file
        dic = jasonReader(path_file)
        slices = dic['slices']
        i = 0
        df = pd.DataFrame(columns=['transient1','transient2'])
        print (file)
        for i in range(0,33):
            try:
                amps = slices[i][param][0:2]
                i += 1
                df.loc[len(df)] = amps
            except IndexError:
                pass
        df.to_csv(path2 + param + '/' + file + param + '.csv', sep='\t')
    print('Done')

# Guarda en una tabla los datos de la célula entera

path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/jsons/'
path2 = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/'
jsons_list = os.listdir(path)
os.makedirs(path2 + 'whole_cell/')
for file in jsons_list:
    path_file = path + file
    dic = jasonReader(path_file)
    dic = dic['image']
    pairs = {k: dic[k] for k in list(dic)[1:]}
    keys = list(pairs.keys())
    df = pd.DataFrame()
    for label in range(0,len(pairs)):
        df[keys[label]] = pairs[keys[label]][0:2]
    df = df.T
    df.columns = ['Transient1', 'Transient2']
    df.to_csv(path2 + 'whole_cell/' + file + '.csv', sep='\t')
print('Done')

In [25]:
# Cálculo del TAU67%


path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/jsons/'
path2 = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/'
jsons_list = os.listdir(path)
# os.makedirs(path2 + 'taus/')
for file in jsons_list:
    path_file = path + file
    dic = jasonReader(path_file)
    dic = dic['slices']
    TAU = pd.DataFrame()
    print(file)
    for peak in range  (0, len(dic)):
        amp = ((np.array(dic[peak]['max_peaks_intensities'][0:2]) - np.array(dic[peak] ['min_peaks_intensities'][1:3]))/np.exp(1))+np.array(dic[peak] ['min_peaks_intensities'][1:3])
        tau_peaks = []
        for i in range(0,len(amp)):
            array = dic[peak]['intensities']
            value = amp[i]
            print (array)
            near_int67 = find_nearest(array, value)
            trans_list = dic[peak]['intensities'][dic[peak]['max_peaks_positions'][i]:dic[peak]['min_peaks_positions'][i+1]]
            tau_peaks.append(trans_list.index(near_int67))
        print (tau_peaks)
#     TAU.to_csv(path2 + 'taus/' + file + '.csv', sep='\t')
    

050320c8_analysis_result_8b000.json
[19, 19, 19, 18, 19, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 19, 20, 22, 24, 26, 28, 30, 33, 36, 39, 42, 45, 47, 49, 51, 53, 55, 56, 57, 57, 57, 56, 56, 55, 55, 54, 54, 53, 53, 52, 51, 51, 50, 50, 49, 48, 48, 47, 47, 46, 46, 45, 45, 45, 44, 44, 43, 42, 42, 42, 41, 41, 40, 40, 39, 39, 39, 39, 38, 38, 37, 37, 37, 37, 36, 36, 36, 36, 35, 35, 35, 35, 34, 34, 34, 33, 33, 33, 33, 32, 32, 32, 31, 31, 31, 30, 30, 30, 30, 30, 29, 29, 29, 28, 28, 28, 28, 28, 27, 27, 27, 26, 26, 26, 26, 26, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 24, 24, 24, 24, 24, 24, 24, 24, 23, 23, 23, 23, 23, 23, 23, 23, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 21, 21, 21, 21, 21, 21, 21, 21, 21, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 18, 18, 18, 18, 18,

ValueError: 64 is not in list

In [ ]:
# Recalcula la amplitud normalizando por el baseline de 1Hz

